In [1]:
import rebound 
import numpy as np
import ctypes
from ctypes import POINTER, c_double, c_int,byref

In [2]:
mercury = ctypes.cdll.LoadLibrary("./mercury/mercury.so")

In [3]:
nbod = c_int(2)
nbig = nbod
algor = 1 # Mixed variable Symplectic
NMESS = 1 # number of messages
CMAX = 1 # number of close encounter minima monitored simultaneously 
step = mercury.mdt_mvsp_
step.argtypes = [
    POINTER(c_double), # real*8              time 
    POINTER(c_double), # real*8              tstart
    POINTER(c_double), # real*8              h0
    POINTER(c_double), # real*8              tol
    POINTER(c_double), # real*8              rmax
    POINTER(c_double*3), # real*8(3)           en
    POINTER(c_double*3), # real*8(3)           am
    POINTER(c_double*3), # real*8(3)           jcen
    POINTER(c_double), # real*8              rcen
    POINTER(c_int), # integer             nbod
    POINTER(c_int), # integer             nbig
    POINTER(c_double*nbod.value), # real*8(nbod)        m
    POINTER((c_double*nbod.value)*3), # real*8(3,nbod)      x
    POINTER((c_double*nbod.value)*3), # real*8(3,nbod)      v
    POINTER((c_double*nbod.value)*3), # real*8(3,nbod)      s
    POINTER(c_double*nbod.value), # real*8(nbod)        rphys
    POINTER(c_double*nbod.value), # real*8(nbod)        rcrit
    POINTER(c_double*nbod.value), # real*8(nbod)        rce
    POINTER(c_int), # integer(nbod)       stat
# character*8(nbod)   id
    POINTER((c_double*nbod.value)*4), # real*8(4,nbod)      ngf
    POINTER(c_int), # integer             algor
    POINTER(c_int*8),# integer(8)          opt
    POINTER(c_int), # integer             dtflag
    POINTER(c_int), # integer             ngflag
    POINTER(c_int), # integer             opflag
    POINTER(c_int), # integer             colflag
    POINTER(c_int), # integer             nclo
    POINTER(c_int*CMAX), # integer(CMAX)       iclo
    POINTER(c_int*CMAX), # integer(CMAX)       jclo
    POINTER(c_double*CMAX), # real*8(CMAX)        dclo
    POINTER(c_double*CMAX), # real*8(CMAX)        tclo
    POINTER(c_double*CMAX), # real*8(CMAX)        ixvclo
    POINTER(c_double*CMAX), # real*8(CMAX)        jxvclo
# character*80(3)     outfile
# character*80(NMESS) mem
    POINTER(c_int*NMESS),# integer(NMESS)     lmem
    ]


In [10]:
time = c_double(0.)
tstart = c_double(0.)
h0 = c_double(0.1)
tol = c_double(1e-6)
rmax = c_double(100.)
DoubleArray3 = c_double * 3
en = DoubleArray3(0.,0.,0.)
am = DoubleArray3(0.,0.,0.)
jcen = DoubleArray3(0.,0.,0.)
rcen = c_double(0.)
DoubleArrayNBOD = c_double * nbod.value
DoubleArrayNBOD3 = (c_double * nbod.value)*3
m = DoubleArrayNBOD(1e-3,1e-3)
x = DoubleArrayNBOD3(DoubleArrayNBOD(1.,2.),DoubleArrayNBOD(0.,0.),DoubleArrayNBOD(0.,0.))
v = DoubleArrayNBOD3(DoubleArrayNBOD(0.,0.),DoubleArrayNBOD(1.,0.5),DoubleArrayNBOD(0.,0.))
s = DoubleArrayNBOD3(DoubleArrayNBOD(0.,0.),DoubleArrayNBOD(0.,0.),DoubleArrayNBOD(0.,0.))
rphys = DoubleArrayNBOD(0.,0.)
rcrit = DoubleArrayNBOD(0.,0.)
rce = DoubleArrayNBOD(0.,0.)
stat = c_int(0)
DoubleArrayNBOD4 = (c_double * nbod.value)*4
ngf = DoubleArrayNBOD4(DoubleArrayNBOD(0,0),DoubleArrayNBOD(0,0),DoubleArrayNBOD(0,0),DoubleArrayNBOD(0,0))
algor = c_int(1)
IntArray8 = c_int*8
opt = IntArray8(0,1,1,2,0,1,0,0)
dtflag = c_int(2)
ngflag = c_int(0)
opflag = c_int(0)
colflag = c_int(0)
nclo = c_int(1)
iclo = (c_int*1)(0)
jclo = (c_int*1)(0)
dclo = (c_double*1)(0.)
tclo = (c_double*1)(0.)
ixvclo = (c_double*1)(0.)
jxvclo = (c_double*1)(0.)
lmem = (c_int*1)(0)

step(byref(time), byref(tstart), byref(h0), byref(tol), byref(rmax), byref(en), byref(am), byref(jcen), byref(rcen),
     byref(nbod), byref(nbig), byref(m), byref(x), byref(v), byref(s), byref(rphys), byref(rcrit), byref(rce), byref(stat),
     byref(ngf), byref(algor), byref(opt), byref(dtflag), byref(ngflag), byref(opflag), byref(colflag), byref(nclo),
     byref(iclo), byref(jclo), byref(dclo), byref(tclo), byref(ixvclo), byref(jxvclo), byref(lmem))

775

In [11]:
x[0][1]

2.0